In [1]:
# Immport
from xml.dom import minidom
import bs4 as bs
import os
from pathlib import Path
import glob
from tqdm import tqdm
import random
import shutil

In [2]:
from collections import Counter
import os
from pathlib import Path
import bs4 as bs

def convertPascal2YOLOv8(filePath, outputDir):
    """
    Pascal VOC formatındaki XML dosyalarını YOLOv8 formatına çevirir
    ID'leri ardışık olacak şekilde yeniden düzenler (0-7)
    """
    # ⚠️ YENİ ARDIIŞIK ID MAPPING (0-7, boşluk yok)
    class_mapping = {
        "D00": 0,     # Longitudinal Crack
        "D10": 1,     # Transverse Crack
        "D20": 2,     # Alligator Crack
        "D40": 3,     # Pothole
        "D43": 4,     # Crosswalk Blur (ESKİ: 6 → YENİ: 4)
        "D44": 5,     # White Line (ESKİ: 7 → YENİ: 5)
        "D50": 6,     # Utility Hole (ESKİ: 8 → YENİ: 6)
        "Repair": 7   # Repair (ESKİ: 9 → YENİ: 7)
    }

    # Sadece bunlar yazılacak
    included_classes = {"D00", "D10", "D20", "D40", "D43", "D44", "D50", "Repair"}

    written_class_counter = Counter()
    ignored_class_counter = Counter()

    try:
        with open(filePath, "r", encoding="utf-8") as file:
            contents = file.read()
    except Exception as e:
        print(f"❌ Dosya okuma hatası: {filePath} - {e}")
        return written_class_counter, ignored_class_counter

    soup = bs.BeautifulSoup(contents, 'lxml-xml')
    
    # Görüntü boyutlarını al
    size_tag = soup.find("size")
    if not size_tag:
        print(f"⚠️ Uyarı: {filePath} dosyasında size tag bulunamadı!")
        return written_class_counter, ignored_class_counter
        
    try:
        image_width = int(size_tag.find("width").text)
        image_height = int(size_tag.find("height").text)
    except (ValueError, AttributeError) as e:
        print(f"⚠️ Görüntü boyut hatası: {filePath} - {e}")
        return written_class_counter, ignored_class_counter

    objects = soup.find_all("object")
    output_lines = []

    for obj in objects:
        try:
            class_name = obj.find("name").text.strip()
        except AttributeError:
            continue

        # Sınıf dahil edilmeyecekse atla
        if class_name not in included_classes:
            ignored_class_counter[class_name] += 1
            continue

        # Mapping'de yoksa atla  
        if class_name not in class_mapping:
            ignored_class_counter[class_name] += 1
            continue

        # YENİ ardışık ID'yi al
        class_id = class_mapping[class_name]

        # Bounding box koordinatlarını al
        try:
            bbox = obj.find("bndbox")
            xmin = float(bbox.find("xmin").text)
            ymin = float(bbox.find("ymin").text)
            xmax = float(bbox.find("xmax").text)
            ymax = float(bbox.find("ymax").text)
        except (ValueError, AttributeError) as e:
            print(f"⚠️ Bbox hatası {class_name}: {e}")
            continue

        # YOLO formatına çevir
        w = xmax - xmin
        h = ymax - ymin
        cx = xmin + w / 2
        cy = ymin + h / 2

        # Normalize et
        w /= image_width
        h /= image_height
        cx /= image_width
        cy /= image_height

        # Geçerli koordinat kontrolü
        if w <= 0 or h <= 0 or cx < 0 or cy < 0 or cx > 1 or cy > 1:
            print(f"⚠️ Geçersiz koordinat {class_name}: cx={cx:.3f}, cy={cy:.3f}, w={w:.3f}, h={h:.3f}")
            continue

        # YENİ ID ile YOLO formatında yaz
        output_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
        written_class_counter[class_name] += 1

    # Çıktı dosyası oluştur (boş olsa bile)
    outputFilename = os.path.splitext(os.path.basename(filePath))[0] + ".txt"
    os.makedirs(outputDir, exist_ok=True)
    outputPath = Path(outputDir) / outputFilename

    with open(outputPath, 'w') as f:
        f.writelines(output_lines)

    return written_class_counter, ignored_class_counter

In [3]:
import glob
from tqdm import tqdm
from collections import Counter

# Ana dizin
ROOTDIR = r"C:\Users\fthgz\OneDrive\Belgeler\RoadDamageDetection-main"

# İşlenecek ülkeler
CountryListDir = [
    "RDD2022_all_countries/Japan/Japan/train/annotations/xmls",
    "RDD2022_all_countries/India/India/train/annotations/xmls",
    "RDD2022_all_countries/China_Drone/China_Drone/train/annotations/xmls",
    "RDD2022_all_countries/China_MotorBike/China_MotorBike/train/annotations/xmls",
    "RDD2022_all_countries/Czech/Czech/train/annotations/xmls",
    "RDD2022_all_countries/Norway/Norway/train/annotations/xmls",
    "RDD2022_all_countries/United_States/United_States/train/annotations/xmls",
]

# Toplam sayaçlar
total_written_counter = Counter()
total_ignored_counter = Counter()

print("🚀 XML dosyalarını YOLO formatına çevirme işlemi başlatılıyor...")
print("="*70)

for country_rel_path in CountryListDir:
    country_abs_path = os.path.join(ROOTDIR, country_rel_path)
    
    # Dizin kontrolü
    if not os.path.exists(country_abs_path):
        print(f"⚠️ Dizin bulunamadı: {country_abs_path}")
        continue
    
    # XML dosyalarını bul
    fileList = sorted(glob.glob(os.path.join(country_abs_path, "*.xml")))
    
    if not fileList:
        print(f"⚠️ {country_rel_path} klasöründe XML dosyası bulunamadı.")
        continue
    
    print(f"\n🔍 {os.path.basename(country_rel_path)} klasöründe {len(fileList)} XML bulundu.")
    
    # Çıktı dizinini belirle (labels klasörü oluştur)
    output_labels_path = Path(country_abs_path).parent.parent / "labels"
    
    # Her XML dosyasını işle
    for file in tqdm(fileList, desc=f"Processing {os.path.basename(country_rel_path)}"):
        written, ignored = convertPascal2YOLOv8(file, output_labels_path)
        total_written_counter.update(written)
        total_ignored_counter.update(ignored)

# İşlem bitince toplu istatistikleri yazdır
print("\n" + "="*70)
print("📊 TÜM DOSYALAR İÇİN TOPLAM SINIF İSTATİSTİKLERİ")
print("="*70)

print("\n✅ YAZILAN SINIFLAR TOPLAM SAYISI:")
if total_written_counter:
    for cls, count in sorted(total_written_counter.items()):
        print(f" - {cls}: {count:,} adet")
    print(f"\n📈 Toplam yazılan nesne sayısı: {sum(total_written_counter.values()):,}")
else:
    print(" Hiç nesne yazılmadı!")

print("\n❌ YAZILMAYAN SINIFLAR TOPLAM SAYISI:")
if total_ignored_counter:
    total_ignored = sum(total_ignored_counter.values())
    print(f"Toplam atlanan nesne sayısı: {total_ignored:,}")
    for cls, count in sorted(total_ignored_counter.items()):
        print(f" - {cls}: {count:,} adet")
else:
    print(" Hiç nesne atlanmadı!")

print(f"\n🎉 XML → YOLO dönüşümü tamamlandı!")
print(f"📁 Label dosyaları her ülkenin 'labels' klasöründe oluşturuldu.")

🚀 XML dosyalarını YOLO formatına çevirme işlemi başlatılıyor...

🔍 xmls klasöründe 10506 XML bulundu.


Processing xmls:  10%|▉         | 1014/10506 [00:12<01:54, 82.92it/s]

⚠️ Geçersiz koordinat D20: cx=0.330, cy=0.791, w=0.000, h=0.002


Processing xmls: 100%|██████████| 10506/10506 [02:14<00:00, 78.22it/s]



🔍 xmls klasöründe 7706 XML bulundu.


...


🔍 xmls klasöründe 8161 XML bulundu.


Processing xmls: 100%|██████████| 8161/8161 [01:40<00:00, 80.99it/s]



🔍 xmls klasöründe 4805 XML bulundu.


Processing xmls: 100%|██████████| 4805/4805 [00:58<00:00, 81.81it/s]


📊 TÜM DOSYALAR İÇİN TOPLAM SINIF İSTATİSTİKLERİ

✅ YAZILAN SINIFLAR TOPLAM SAYISI:
 - D00: 26,016 adet
 - D10: 11,830 adet
 - D20: 10,616 adet
 - D40: 6,544 adet
 - D43: 793 adet
 - D44: 5,057 adet
 - D50: 3,581 adet
 - Repair: 1,046 adet

📈 Toplam yazılan nesne sayısı: 65,483

❌ YAZILMAYAN SINIFLAR TOPLAM SAYISI:
Toplam atlanan nesne sayısı: 228
 - Block crack: 3 adet
 - D01: 179 adet
 - D0w0: 1 adet
 - D11: 45 adet

🎉 XML → YOLO dönüşümü tamamlandı!
📁 Label dosyaları her ülkenin 'labels' klasöründe oluşturuldu.


In [4]:
import os
import glob
import random
import shutil
from tqdm import tqdm
from pathlib import Path

def CopyDatasetSplit(baseDir):
    """
    Veri setini %80 eğitim, %20 doğrulama olarak böler ve kopyalar
    Boş etiketli görüntüleri %5 oranında dahil eder
    """
    random.seed(1337)  # Reproducible results
    
    baseOutputDir = r"C:\Users\fthgz\OneDrive\Belgeler\RoadDamageDetection-main\RDD2022_SPLIT"
    
    baseImageDir = os.path.join(baseDir, "images")
    baseAnnotDir = os.path.join(baseDir, "labels")
    
    # Dizin kontrolü
    if not os.path.exists(baseImageDir):
        print(f"⚠️ Images dizini bulunamadı: {baseImageDir}")
        return
        
    if not os.path.exists(baseAnnotDir):
        print(f"⚠️ Labels dizini bulunamadı: {baseAnnotDir}")
        return
    
    # Tüm dosyaları listele
    image_list_all = sorted(glob.glob(os.path.join(baseImageDir, "*")))
    annot_list_all = sorted(glob.glob(os.path.join(baseAnnotDir, "*")))
    
    if not image_list_all:
        print(f"⚠️ Hiç görüntü dosyası bulunamadı: {baseImageDir}")
        return
        
    if not annot_list_all:
        print(f"⚠️ Hiç etiket dosyası bulunamadı: {baseAnnotDir}")
        return
    
    print(f"📊 Bulunan dosyalar - Görüntü: {len(image_list_all)}, Etiket: {len(annot_list_all)}")
    
    # 🟡 Boş etiketli görüntüler %5 oranında dahil edilecek
    backgroundImages_Percentage = 0.05
    image_list = []
    annot_list = []
    max_background_image = int(len(image_list_all) * backgroundImages_Percentage)
    background_counter = 0
    files_with_labels = 0
    
    # Dosya eşleştirmesi ve filtreleme
    min_length = min(len(image_list_all), len(annot_list_all))
    
    for i in range(min_length):
        try:
            with open(annot_list_all[i], 'r', encoding='utf-8') as f:
                annot_content = f.read().strip()
                
            if annot_content:  # Etiket varsa
                image_list.append(image_list_all[i])
                annot_list.append(annot_list_all[i])
                files_with_labels += 1
            elif background_counter < max_background_image:  # Boş etiket ama limitte
                image_list.append(image_list_all[i])
                annot_list.append(annot_list_all[i])
                background_counter += 1
                
        except Exception as e:
            print(f"⚠️ Dosya okuma hatası: {annot_list_all[i]} - {e}")
            continue
    
    if not image_list:
        print("⚠️ İşlenecek dosya bulunamadı!")
        return
    
    print(f"📈 Etiketli dosyalar: {files_with_labels}")
    print(f"📉 Boş etiketli dosyalar (dahil edilen): {background_counter}")
    print(f"📊 Toplam işlenecek dosya: {len(image_list)}")
    
    # 🔀 %80 eğitim, %20 doğrulama
    dataset_length = len(image_list)
    split_ratio = 0.8
    middle_point = round(split_ratio * dataset_length)
    
    numberList = list(range(dataset_length))
    random.shuffle(numberList)
    trainNumberList = numberList[:middle_point]
    validNumberList = numberList[middle_point:]
    
    print(f"📊 Toplam Eğitim/Doğrulama Sayısı: {len(trainNumberList)} / {len(validNumberList)}")
    
    # 🟩 Eğitim verileri
    print("📥 Eğitim verileri kopyalanıyor...")
    train_images_dir = os.path.join(baseOutputDir, "images", "train")
    train_labels_dir = os.path.join(baseOutputDir, "labels", "train")
    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    
    for i in tqdm(trainNumberList, desc="Train Copy"):
        try:
            shutil.copy2(image_list[i], train_images_dir)
            shutil.copy2(annot_list[i], train_labels_dir)
        except Exception as e:
            print(f"⚠️ Eğitim kopyalama hatası: {e}")
    
    # 🟦 Doğrulama verileri
    print("📥 Doğrulama verileri kopyalanıyor...")
    val_images_dir = os.path.join(baseOutputDir, "images", "val")
    val_labels_dir = os.path.join(baseOutputDir, "labels", "val")
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)
    
    for i in tqdm(validNumberList, desc="Val Copy"):
        try:
            shutil.copy2(image_list[i], val_images_dir)
            shutil.copy2(annot_list[i], val_labels_dir)
        except Exception as e:
            print(f"⚠️ Doğrulama kopyalama hatası: {e}")
    
    print(f"✅ {os.path.basename(baseDir)} için dataset split tamamlandı!")

In [5]:
import os

# Ana dizin
ROOTDIR = r"C:\Users\fthgz\OneDrive\Belgeler\RoadDamageDetection-main"

# İşlenecek ülkeler
COUNTRIES = ["Japan", "India", "China_MotorBike", "China_Drone", "Czech", "Norway", "United_States"]

print("🌍 Tüm ülkeler için dataset split işlemi başlatılıyor...")
print("="*60)

successful_countries = []
failed_countries = []

for country in COUNTRIES:
    # Dizin yapısı: RDD2022_all_countries/{country}/{country}/train
    base_train_dir = os.path.join(ROOTDIR, "RDD2022_all_countries", country, country, "train")
    
    if os.path.exists(base_train_dir):
        print(f"\n🚀 İşleniyor: {country}")
        print("-" * 40)
        try:
            CopyDatasetSplit(base_train_dir)
            successful_countries.append(country)
            print(f"✅ {country} başarıyla tamamlandı!")
        except Exception as e:
            print(f"❌ {country} işlenirken hata: {e}")
            failed_countries.append(country)
    else:
        print(f"⚠️ Dizin bulunamadı: {base_train_dir}")
        failed_countries.append(country)

# Özet rapor
print("\n" + "="*60)
print("📋 İŞLEM ÖZETİ")
print("="*60)

print(f"\n✅ BAŞARILI ÜLKELER ({len(successful_countries)}):")
for country in successful_countries:
    print(f"   - {country}")

if failed_countries:
    print(f"\n❌ BAŞARISIZ ÜLKELER ({len(failed_countries)}):")
    for country in failed_countries:
        print(f"   - {country}")

print(f"\n🎉 Dataset split işlemi tamamlandı!")
print(f"📁 Çıktı dizini: {ROOTDIR}\\RDD2022_SPLIT")
print("\n📊 Final dataset yapısı:")
print("RDD2022_SPLIT/")
print("├── images/")
print("│   ├── train/")
print("│   └── val/")
print("└── labels/")
print("    ├── train/")
print("    └── val/")

🌍 Tüm ülkeler için dataset split işlemi başlatılıyor...

🚀 İşleniyor: Japan
----------------------------------------
📊 Bulunan dosyalar - Görüntü: 10506, Etiket: 10506
📈 Etiketli dosyalar: 9712
📉 Boş etiketli dosyalar (dahil edilen): 525
📊 Toplam işlenecek dosya: 10237
📊 Toplam Eğitim/Doğrulama Sayısı: 8190 / 2047
📥 Eğitim verileri kopyalanıyor...


Train Copy: 100%|██████████| 8190/8190 [01:44<00:00, 78.68it/s]


📥 Doğrulama verileri kopyalanıyor...


Val Copy: 100%|██████████| 2047/2047 [00:26<00:00, 76.69it/s]


✅ train için dataset split tamamlandı!
✅ Japan başarıyla tamamlandı!

🚀 İşleniyor: India
----------------------------------------
📊 Bulunan dosyalar - Görüntü: 7706, Etiket: 7706
📈 Etiketli dosyalar: 3701
📉 Boş etiketli dosyalar (dahil edilen): 385
📊 Toplam işlenecek dosya: 4086
📊 Toplam Eğitim/Doğrulama Sayısı: 3269 / 817
📥 Eğitim verileri kopyalanıyor...


...

✅ train için dataset split tamamlandı!
✅ Norway başarıyla tamamlandı!

🚀 İşleniyor: United_States
----------------------------------------
📊 Bulunan dosyalar - Görüntü: 4805, Etiket: 4805
📈 Etiketli dosyalar: 4805
📉 Boş etiketli dosyalar (dahil edilen): 0
📊 Toplam işlenecek dosya: 4805
📊 Toplam Eğitim/Doğrulama Sayısı: 3844 / 961
📥 Eğitim verileri kopyalanıyor...


Train Copy: 100%|██████████| 3844/3844 [00:52<00:00, 72.71it/s]


📥 Doğrulama verileri kopyalanıyor...


Val Copy: 100%|██████████| 961/961 [00:12<00:00, 74.41it/s]

✅ train için dataset split tamamlandı!
✅ United_States başarıyla tamamlandı!

📋 İŞLEM ÖZETİ

✅ BAŞARILI ÜLKELER (7):
   - Japan
   - India
   - China_MotorBike
   - China_Drone
   - Czech
   - Norway
   - United_States

🎉 Dataset split işlemi tamamlandı!
📁 Çıktı dizini: C:\Users\fthgz\OneDrive\Belgeler\RoadDamageDetection-main\RDD2022_SPLIT

📊 Final dataset yapısı:
RDD2022_SPLIT/
├── images/
│   ├── train/
│   └── val/
└── labels/
    ├── train/
    └── val/


In [8]:
!tree ./ -d

Too many parameters - -d
